In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import datetime
import matplotlib.pyplot as plt

In [3]:
df_depression = pd.read_csv('depression_posts_scraped.csv')
print(df_depression.shape)
df_depression.head(1)

(4035, 13)


title  score    p_id   subreddit  \
0  Regular Check-In Post    970  cml6ni  depression   

                                                 url  num_comments  \
0  https://www.reddit.com/r/depression/comments/c...          5523   

                                                body          p_timestamp  \
0  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   

      c_id                                            comment  \
0  ew4joli  I gave in to my depression and stayed home ins...   

           c_timestamp Post_Reply            Time_to_Comment  
0  2019-08-06 20:21:04          Y  0 days 14:08:31.000000000

In [15]:
# Top 10 comments by score
list(df_depression.sort_values(by = ['score'], ascending = False)['comment'][:10])

["I know this is cliche and I'm sure you are tired of hearing about it, but it really does get better. A lot of it is mentality. I don't know what you've gone through but one thing I've learned is that it doesn't really matter. You can have a seemingly perfect life but you can still be depressed. Other people can go through hell and not be depressed at all. It's how your brain is wired which can be affected by both genetics and you're environment. It may not be possible right now, but work on getting in a better environment, maybe get a fresh start. Talk it out with people and just try your absolute hardest to think more positive. Instead of focusing on the bad, focus on the good. I'm not saying you'll be perfectly fine all of a sudden because depression sucks, but it's a step in the right direction. Please don't give up and if you are myopically focused on getting better, you will see results. Good luck, hang in there.",
 "Honestly same. I want to die so much but than again I don't ha

In [16]:
# Top 10 comments by number of comments
list(df_depression.sort_values(by = ['num_comments'], ascending = False)['comment'][:10])

["I gave in to my depression and stayed home instead of going to an exam that made up 35% of my grade. Then I thought to myself what the fuck did I just do and I dragged my ass to office hours to beg for another chance and told my professor honestly why I didn't attend. \n\nHe was very sympathetic and told me he could tell I was having a hard time and said I could take it tomorrow. I don't feel like I deserve this chance, but I won't waste it.",
 'Doing shitty. I wanna die .',
 "I feel really guilty for having to step down from management to part time. I feel like a failure, and that I slowly lead everyone into the ground. I stepped down because my depression is getting bad, and I'm trying to get on disability, but my husband is getting so overwhelmingly stressed out that I feel even more guilty for stepping down. I feel like a burden. If I never stepped down, this never would have happened. But if I didn't step down, I would have had a total mental breakdown- I already had a minor one

In [20]:
df_depression_post = df_depression[['p_id', 'score', 'num_comments', 'p_timestamp']].drop_duplicates(keep='first')
df_depression_post.head()

p_id  score  num_comments          p_timestamp
0    cml6ni    970          5523  2019-08-06 06:12:33
470  doqwow    458            45  2019-10-29 17:52:02
518  dzkwym    207            37  2019-11-21 17:32:06
555  dzg28a    543            64  2019-11-21 10:00:35
618  dzhsg2    115            13  2019-11-21 13:10:35

In [19]:
df_depression_post_direct_reply = df_depression[df_depression['Post_Reply']=='Y'].groupby('p_id')['num_comments'].count()
df_depression_post_direct_reply = df_depression_post_direct_reply.to_frame().rename(columns={'num_comments': 'direct_reply_comments'})
df_depression_post = pd.merge(df_depression_post, df_depression_post_direct_reply, how = 'left', on = 'p_id')

df_depression_post['direct_comments_proportion'] = df_depression_post['direct_reply_comments']/df_depression_post['num_comments']
df_depression_post['direct_comments_proportion'] = df_depression_post['direct_comments_proportion'].fillna(0)

df_depression_post.head()

p_id  score  num_comments          p_timestamp  direct_reply_comments  \
0  cml6ni    970          5523  2019-08-06 06:12:33                  443.0   
1  doqwow    458            45  2019-10-29 17:52:02                   20.0   
2  dzkwym    207            37  2019-11-21 17:32:06                   29.0   
3  dzg28a    543            64  2019-11-21 10:00:35                   49.0   
4  dzhsg2    115            13  2019-11-21 13:10:35                   11.0   

   direct_comments_proportion  
0                    0.080210  
1                    0.444444  
2                    0.783784  
3                    0.765625  
4                    0.846154

In [22]:
def calc_minutes(s):
    try:
        s = s.lstrip()
        days = int(s.split(' ', 1)[0])
        s = s.split(' ', 1)[1].lstrip().split(' ', 1)[1] + ' '
        hours = int(s[0:2])
        minutes = int(s[3:5])
        seconds = int(s[6:8])
    
        return days*1440 + hours*60 + minutes + seconds/60
    except:
        return np.NaN


In [23]:
df_depression_post_direct_reply_time = df_depression[df_depression['Post_Reply']=='Y']
df_depression_post_direct_reply_time.head()

df_depression_post_direct_reply_time['time_to_reply'] = 0

for i in range(len(df_depression_post_direct_reply_time)):
    df_depression_post_direct_reply_time['time_to_reply'].iloc[i] = calc_minutes(df_depression_post_direct_reply_time['Time_to_Comment'].iloc[i])


                                                                                      
df_depression_post_direct_reply_time = df_depression_post_direct_reply_time.groupby('p_id')['time_to_reply'].aggregate(['median', 'min'])

df_depression_post = pd.merge(df_depression_post, df_depression_post_direct_reply_time, how = 'left', on = 'p_id')
df_depression_post = df_depression_post.rename(columns={'median': 'median_direct_reply_time', 'min': 'min_reply_time'})

df_depression_post.head()

C:\Users\Hannah\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Hannah\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Hannah\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/in

p_id  score  num_comments          p_timestamp  median_direct_reply_time  \
0  cml6ni    970          5523  2019-08-06 06:12:33              43287.350000   
1  doqwow    458            45  2019-10-29 17:52:02               8601.425000   
2  dzkwym    207            37  2019-11-21 17:32:06                106.650000   
3  dzg28a    543            64  2019-11-21 10:00:35                523.966667   
4  dzhsg2    115            13  2019-11-21 13:10:35                340.600000   

   min_reply_time  
0        5.933333  
1       51.350000  
2       15.666667  
3        1.100000  
4       10.366667

In [24]:
sw = stopwords.words('english')

df_depression['body'] = df_depression['body'].astype(str).str.replace('\n', '').str.replace('\\', '')

df_depression['body_sw'] = df_depression['body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)])).astype(str)

#remove special characters, make all characters lowercase
df_depression['body_sw_p'] = df_depression['body_sw'].str.replace('\w\s]', '').str.lower()

df_depression['body_length'] = 0

for i in range(len(df_depression)):
    df_depression['body_length'].iloc[i] = len(df_depression['body'].iloc[i].split(' '))



In [26]:
analyser = SentimentIntensityAnalyzer()

df_depression['Sentiment Scores'] = df_depression['body_sw'].apply(analyser.polarity_scores)
df_depression['P_Sent_Neg'] = np.NaN
df_depression['P_Sent_Neu'] = np.NaN
df_depression['P_Sent_Pos'] = np.NaN
df_depression['P_Sent_Com'] = np.NaN


for i in range(len(df_depression)):
    df_depression['P_Sent_Neg'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['neg']
    df_depression['P_Sent_Neu'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['neu']
    df_depression['P_Sent_Pos'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['pos']
    df_depression['P_Sent_Com'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['compound']

    

df_depression_post_sentiments = df_depression.groupby('p_id')[['P_Sent_Neg', 'P_Sent_Neu', 'P_Sent_Pos', 'P_Sent_Com']].mean()

df_depression_post = pd.merge(df_depression_post, df_depression_post_sentiments, how = 'left', on = 'p_id')
df_depression_post.head()

p_id  score  num_comments          p_timestamp  median_direct_reply_time  \
0  cml6ni    970          5523  2019-08-06 06:12:33              43287.350000   
1  doqwow    458            45  2019-10-29 17:52:02               8601.425000   
2  dzkwym    207            37  2019-11-21 17:32:06                106.650000   
3  dzg28a    543            64  2019-11-21 10:00:35                523.966667   
4  dzhsg2    115            13  2019-11-21 13:10:35                340.600000   

   min_reply_time  P_Sent_Neg  P_Sent_Neu  P_Sent_Pos  P_Sent_Com  
0        5.933333       0.063       0.626       0.312      0.9493  
1       51.350000       0.119       0.623       0.259      0.9967  
2       15.666667       0.424       0.424       0.153     -0.8625  
3        1.100000       0.122       0.556       0.322      0.8560  
4       10.366667       0.173       0.577       0.250      0.7269

In [59]:
df_depression['comment'] = df_depression['comment'].astype(str).str.replace('\n', '').str.replace('\\', '')

df_depression['comment_sw'] = df_depression['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)])).astype(str)

#remove special characters, make all characters lowercase
df_depression['comment_sw_p'] = df_depression['comment_sw'].str.replace('[^\w\s]', '').str.lower()


#Create boolean for deleted comments
df_depression['deleted'] = np.where(df_depression['comment']=='[deleted]', 'Y', 'N')

df_depression['comment_length'] = 0

for i in range(len(df_depression)):
    df_depression['comment_length'].iloc[i] = len(df_depression['comment'].iloc[i].split(' '))

df_depression['comment_length'] = np.where(df_depression['deleted']=='Y', np.NaN, df_depression['comment_length'])

In [60]:
# Create series of comment length per user and merge with main dataframe
df_depression_comment_length = df_depression.groupby('p_id')['comment_length'].median()
df_depression_comment_length = df_depression_comment_length.to_frame().rename(columns={'comment_length': 'median_comment_length'})

df_depression_post = pd.merge(df_depression_post, df_depression_comment_length, how = 'left', on = 'p_id')
df_depression_post.head()

p_id  score  num_comments          p_timestamp  direct_reply_comments  \
0  cml6ni    970          5523  2019-08-06 06:12:33                  443.0   
1  doqwow    458            45  2019-10-29 17:52:02                   20.0   
2  dzkwym    207            37  2019-11-21 17:32:06                   29.0   
3  dzg28a    543            64  2019-11-21 10:00:35                   49.0   
4  dzhsg2    115            13  2019-11-21 13:10:35                   11.0   

   direct_comments_proportion  median_direct_reply_time  min_reply_time  \
0                    0.080210              43287.350000        5.933333   
1                    0.444444               8601.425000       51.350000   
2                    0.783784                106.650000       15.666667   
3                    0.765625                523.966667        1.100000   
4                    0.846154                340.600000       10.366667   

   P_Sent_Neg  P_Sent_Neu  P_Sent_Pos  P_Sent_Com  median_comment_length  
0       0.063       0.626       0.312      0.9493                   44.0  
1       0.119       0.623       0.259      0.9967                   30.5  
2       0.424       0.424       0.153     -0.8625                   33.0  
3       0.122       0.556       0.322      0.8560                   24.0  
4       0.173       0.577       0.250      0.7269                   52.0

In [61]:
df_depression['Sentiment Scores'] = df_depression['comment_sw'].apply(analyser.polarity_scores)
df_depression['C_Sent_Neg'] = np.NaN
df_depression['C_Sent_Neu'] = np.NaN
df_depression['C_Sent_Pos'] = np.NaN
df_depression['C_Sent_Com'] = np.NaN


for i in range(len(df_depression)):
    df_depression['C_Sent_Neg'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['neg']
    df_depression['C_Sent_Neu'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['neu']
    df_depression['C_Sent_Pos'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['pos']
    df_depression['C_Sent_Com'].iloc[i] = df_depression['Sentiment Scores'].iloc[i]['compound']

df_depression['C_Sent_Neg'] = np.where(df_depression['comment']=='[deleted]', np.NaN, df_depression['C_Sent_Neg'])
df_depression['C_Sent_Neu'] = np.where(df_depression['comment']=='[deleted]', np.NaN, df_depression['C_Sent_Neu'])
df_depression['C_Sent_Pos'] = np.where(df_depression['comment']=='[deleted]', np.NaN, df_depression['C_Sent_Pos'])
df_depression['C_Sent_Com'] = np.where(df_depression['comment']=='[deleted]', np.NaN, df_depression['C_Sent_Com'])
    
    
df_depression_comment_sentiments = df_depression.groupby('p_id')[['C_Sent_Neg', 'C_Sent_Neu', 'C_Sent_Pos', 'C_Sent_Com']].median()

df_depression_post = pd.merge(df_depression_post, df_depression_comment_sentiments, how = 'left', on = 'p_id')
df_depression_post.head()

p_id  score  num_comments          p_timestamp  direct_reply_comments  \
0  cml6ni    970          5523  2019-08-06 06:12:33                  443.0   
1  doqwow    458            45  2019-10-29 17:52:02                   20.0   
2  dzkwym    207            37  2019-11-21 17:32:06                   29.0   
3  dzg28a    543            64  2019-11-21 10:00:35                   49.0   
4  dzhsg2    115            13  2019-11-21 13:10:35                   11.0   

   direct_comments_proportion  median_direct_reply_time  min_reply_time  \
0                    0.080210              43287.350000        5.933333   
1                    0.444444               8601.425000       51.350000   
2                    0.783784                106.650000       15.666667   
3                    0.765625                523.966667        1.100000   
4                    0.846154                340.600000       10.366667   

   P_Sent_Neg  P_Sent_Neu  P_Sent_Pos  P_Sent_Com  median_comment_length  \
0       0.063       0.626       0.312      0.9493                   44.0   
1       0.119       0.623       0.259      0.9967                   30.5   
2       0.424       0.424       0.153     -0.8625                   33.0   
3       0.122       0.556       0.322      0.8560                   24.0   
4       0.173       0.577       0.250      0.7269                   52.0   

   C_Sent_Neg  C_Sent_Neu  C_Sent_Pos  C_Sent_Com  
0      0.2120      0.5690      0.1875     -0.1393  
1      0.0720      0.6045      0.2505      0.4490  
2      0.1810      0.5380      0.1690      0.0000  
3      0.1870      0.5820      0.1990      0.0000  
4      0.1675      0.5875      0.2705      0.3066

In [62]:
df_depression_post['Diff_Sent_Neg'] = df_depression_post['C_Sent_Neg'] - df_depression_post['P_Sent_Neg']
df_depression_post['Diff_Sent_Neu'] = df_depression_post['C_Sent_Neu'] - df_depression_post['P_Sent_Neu']
df_depression_post['Diff_Sent_Pos'] = df_depression_post['C_Sent_Pos'] - df_depression_post['P_Sent_Pos']
df_depression_post['Diff_Sent_Com'] = df_depression_post['C_Sent_Com'] - df_depression_post['P_Sent_Com']

df_depression_post.head()

p_id  score  num_comments          p_timestamp  direct_reply_comments  \
0  cml6ni    970          5523  2019-08-06 06:12:33                  443.0   
1  doqwow    458            45  2019-10-29 17:52:02                   20.0   
2  dzkwym    207            37  2019-11-21 17:32:06                   29.0   
3  dzg28a    543            64  2019-11-21 10:00:35                   49.0   
4  dzhsg2    115            13  2019-11-21 13:10:35                   11.0   

   direct_comments_proportion  median_direct_reply_time  min_reply_time  \
0                    0.080210              43287.350000        5.933333   
1                    0.444444               8601.425000       51.350000   
2                    0.783784                106.650000       15.666667   
3                    0.765625                523.966667        1.100000   
4                    0.846154                340.600000       10.366667   

   P_Sent_Neg  P_Sent_Neu  ...  P_Sent_Com  median_comment_length  C_Sent_Neg  \
0       0.063       0.626  ...      0.9493                   44.0      0.2120   
1       0.119       0.623  ...      0.9967                   30.5      0.0720   
2       0.424       0.424  ...     -0.8625                   33.0      0.1810   
3       0.122       0.556  ...      0.8560                   24.0      0.1870   
4       0.173       0.577  ...      0.7269                   52.0      0.1675   

   C_Sent_Neu  C_Sent_Pos  C_Sent_Com  Diff_Sent_Neg  Diff_Sent_Neu  \
0      0.5690      0.1875     -0.1393         0.1490        -0.0570   
1      0.6045      0.2505      0.4490        -0.0470        -0.0185   
2      0.5380      0.1690      0.0000        -0.2430         0.1140   
3      0.5820      0.1990      0.0000         0.0650         0.0260   
4      0.5875      0.2705      0.3066        -0.0055         0.0105   

   Diff_Sent_Pos  Diff_Sent_Com  
0        -0.1245        -1.0886  
1        -0.0085        -0.5477  
2         0.0160         0.8625  
3        -0.1230        -0.8560  
4         0.0205        -0.4203  

[5 rows x 21 columns]

In [63]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [64]:
df_depression['body_lemmatized'] = ''
df_depression['comment_lemmatized'] = ''

for i in range(len(df_depression)):
    df_depression['body_lemmatized'].iloc[i] = lemmatize_text(df_depression['body_sw_p'].iloc[i])
    df_depression['comment_lemmatized'].iloc[i] = lemmatize_text(df_depression['comment_sw_p'].iloc[i])

In [65]:
neg_feelings = ['sad', 'bitter', 'regret', 'hate', 'hopeless', 'exhausted', 'numb', 'tired',
                'depressed', 'alone', 'lonely', 'isolate', 'shitty', 'failure', 'cry',
                'worthless', 'empty', 'toxic', 'prison', 'torture', 'boring', 'monotonous',
                'sucks', 'pointless', 'nothing', 'unhappy', 'meaningless', 'anxiety', 'invisibile',
                'abusive', 'struggle']

suicide_act = ['kill', 'die', 'rope', 'knife', 'pills', 'hang', 'cut', 'suicide']

goal = ['motivation', 'care', 'achieve', 'happy', 'strong', 'proud', 'socialize', 'hope',
        'excited', 'bath', 'shower', 'family', 'help', 'improvement', 'flush']

medical = ['antidepressant', 'therapy', 'therapy', 'therapist', 'psychiatrist', 'medicate', 'medicine']

fps = ['i', "i'm", 'im', 'me', 'myself', 'my']



neg_feelings = " ".join(neg_feelings)
suicide_act = " ".join(suicide_act)
goal = " ".join(goal)
medical = " ".join(medical)
fps = " ".join(fps)

In [66]:
df_depression['C_Neg_Feelings'] = 0 
df_depression['C_Suicide_Act'] = 0
df_depression['C_Goal'] = 0
df_depression['C_Medical'] = 0
df_depression['C_FPS'] = 0

df_depression['P_Neg_Feelings'] = 0 
df_depression['P_Suicide_Act'] = 0
df_depression['P_Goal'] = 0
df_depression['P_Medical'] = 0
df_depression['P_FPS'] = 0

In [67]:
cols_c = ['C_Neg_Feelings', 'C_Suicide_Act', 'C_Goal', 'C_Medical', 'C_FPS']
cols_p = ['P_Neg_Feelings', 'P_Suicide_Act', 'P_Goal', 'P_Medical', 'P_FPS']

lists = [neg_feelings, suicide_act, goal, medical, fps]


for i in range(len(cols_c)):
    for j in range(len(df_depression['comment_lemmatized'])):
        
        count = 0
        for k in range(len(df_depression['comment_lemmatized'].iloc[j])):
            if(df_depression['comment_lemmatized'].iloc[j][k].lower() in lists[i]):
                count += 1
        if(len(df_depression['comment_lemmatized'].iloc[j])==0):
            df_depression[cols_c[i]].iloc[j] = 0
        else:
            df_depression[cols_c[i]].iloc[j] = count/len(df_depression['comment_lemmatized'].iloc[j])

            
for i in range(len(cols_p)):
    for j in range(len(df_depression['body_lemmatized'])):
        
        count = 0
        for k in range(len(df_depression['body_lemmatized'].iloc[j])):
            if(df_depression['body_lemmatized'].iloc[j][k].lower() in lists[i]):
                count += 1
        if(len(df_depression['body_lemmatized'].iloc[j])==0):
            df_depression[cols_p[i]].iloc[j] = 0
        else:
            df_depression[cols_p[i]].iloc[j] = count/len(df_depression['body_lemmatized'].iloc[j])

In [68]:
df_depression_frequencies = df_depression.groupby('p_id')['C_Neg_Feelings', 'C_Suicide_Act', 'C_Goal', 
                                                          'C_Medical', 'C_FPS', 'P_Neg_Feelings', 'P_Suicide_Act', 
                                                          'P_Goal', 'P_Medical', 'P_FPS'].median()

df_depression_post = pd.merge(df_depression_post, df_depression_frequencies, how = 'left', on = 'p_id')

In [69]:
df_depression_post.head(5)

p_id  score  num_comments          p_timestamp  direct_reply_comments  \
0  cml6ni    970          5523  2019-08-06 06:12:33                  443.0   
1  doqwow    458            45  2019-10-29 17:52:02                   20.0   
2  dzkwym    207            37  2019-11-21 17:32:06                   29.0   
3  dzg28a    543            64  2019-11-21 10:00:35                   49.0   
4  dzhsg2    115            13  2019-11-21 13:10:35                   11.0   
5  dzndnc     25             8  2019-11-21 20:17:15                    5.0   
6  dzj6rf     68             3  2019-11-21 15:17:38                    1.0   
7  dzj8z4     62            21  2019-11-21 15:22:46                   13.0   
8  dzlltw     34            18  2019-11-21 18:17:46                   12.0   
9  dzg1iq    156            19  2019-11-21 09:58:05                   12.0   

   direct_comments_proportion  median_direct_reply_time  min_reply_time  \
0                    0.080210              43287.350000        5.933333   
1                    0.444444               8601.425000       51.350000   
2                    0.783784                106.650000       15.666667   
3                    0.765625                523.966667        1.100000   
4                    0.846154                340.600000       10.366667   
5                    0.625000                193.600000        4.783333   
6                    0.333333                  4.883333        4.883333   
7                    0.619048                132.733333        1.066667   
8                    0.666667                117.125000        0.866667   
9                    0.631579                379.066667        3.150000   

   P_Sent_Neg  P_Sent_Neu  ...  C_Neg_Feelings  C_Suicide_Act    C_Goal  \
0       0.063       0.626  ...        0.166667       0.098269  0.156438   
1       0.119       0.623  ...        0.123724       0.032522  0.083333   
2       0.424       0.424  ...        0.130435       0.046512  0.115385   
3       0.122       0.556  ...        0.185185       0.055556  0.153846   
4       0.173       0.577  ...        0.169540       0.110976  0.208711   
5       0.209       0.791  ...        0.106667       0.000000  0.076255   
6       0.431       0.384  ...        0.070197       0.052340  0.070197   
7       0.332       0.448  ...        0.125000       0.000000  0.100000   
8       0.327       0.432  ...        0.158333       0.052778  0.139610   
9       0.000       0.435  ...        0.094017       0.000000  0.000000   

   C_Medical     C_FPS  P_Neg_Feelings  P_Suicide_Act    P_Goal  P_Medical  \
0   0.107143  0.126555        0.000000       0.033898  0.000000   0.016949   
1   0.028665  0.021779        0.016317       0.013986  0.025641   0.002331   
2   0.000000  0.064516        0.045455       0.045455  0.090909   0.000000   
3   0.083333  0.092593        0.179487       0.051282  0.076923   0.051282   
4   0.144659  0.144309        0.194030       0.149254  0.179104   0.134328   
5   0.000000  0.071429        0.000000       0.000000  0.000000   0.058824   
6   0.114840  0.043411        0.200000       0.066667  0.133333   0.066667   
7   0.000000  0.000000        0.058824       0.000000  0.000000   0.000000   
8   0.052778  0.118056        0.083333       0.083333  0.083333   0.041667   
9   0.000000  0.000000        0.333333       0.333333  0.333333   0.333333   

      P_FPS  
0  0.000000  
1  0.000000  
2  0.045455  
3  0.076923  
4  0.149254  
5  0.000000  
6  0.133333  
7  0.000000  
8  0.083333  
9  0.333333  

[10 rows x 31 columns]

In [70]:
df_depression_post.shape

(934, 31)

In [71]:
df_depression.head(5)

title  score    p_id   subreddit  \
0  Regular Check-In Post    970  cml6ni  depression   
1  Regular Check-In Post    970  cml6ni  depression   
2  Regular Check-In Post    970  cml6ni  depression   
3  Regular Check-In Post    970  cml6ni  depression   
4  Regular Check-In Post    970  cml6ni  depression   
5  Regular Check-In Post    970  cml6ni  depression   
6  Regular Check-In Post    970  cml6ni  depression   
7  Regular Check-In Post    970  cml6ni  depression   
8  Regular Check-In Post    970  cml6ni  depression   
9  Regular Check-In Post    970  cml6ni  depression   

                                                 url  num_comments  \
0  https://www.reddit.com/r/depression/comments/c...          5523   
1  https://www.reddit.com/r/depression/comments/c...          5523   
2  https://www.reddit.com/r/depression/comments/c...          5523   
3  https://www.reddit.com/r/depression/comments/c...          5523   
4  https://www.reddit.com/r/depression/comments/c...          5523   
5  https://www.reddit.com/r/depression/comments/c...          5523   
6  https://www.reddit.com/r/depression/comments/c...          5523   
7  https://www.reddit.com/r/depression/comments/c...          5523   
8  https://www.reddit.com/r/depression/comments/c...          5523   
9  https://www.reddit.com/r/depression/comments/c...          5523   

                                                body          p_timestamp  \
0  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
1  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
2  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
3  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
4  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
5  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
6  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
7  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
8  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   
9  Welcome to /r/depression's check-in post - a p...  2019-08-06 06:12:33   

      c_id                                            comment  ...  \
0  ew4joli  I gave in to my depression and stayed home ins...  ...   
1  ewejn04  Currently experiencing: Extreme sadness (but c...  ...   
2  ew6orjt  i’m just too scared to kill myself. i wish i c...  ...   
3  ew3efn2  I feel lonely. I have isolated myself from my ...  ...   
4  ew74yn6  I opened my blinds today and showered. I haven...  ...   
5  exjxu2k  Ever felt so alone even when always surrounded...  ...   
6  ew6ajdu  i’m afraid of not fulfilling potentiali think ...  ...   
7  eyf87xw  Today was my second day of classes. In my soci...  ...   
8  ew48vd4  I'm starting to realize that my brain is toxic...  ...   
9  ew5buhm                                          [deleted]  ...   

  C_Neg_Feelings C_Suicide_Act    C_Goal C_Medical     C_FPS  P_Neg_Feelings  \
0       0.215686      0.196078  0.235294  0.215686  0.196078             0.0   
1       0.266667      0.066667  0.133333  0.100000  0.100000             0.0   
2       0.000000      0.222222  0.111111  0.000000  0.222222             0.0   
3       0.359375      0.234375  0.265625  0.250000  0.250000             0.0   
4       0.080000      0.080000  0.160000  0.080000  0.160000             0.0   
5       0.142857      0.000000  0.000000  0.000000  0.000000             0.0   
6       0.105263      0.000000  0.105263  0.000000  0.105263             0.0   
7       0.140496      0.074380  0.107438  0.082645  0.082645             0.0   
8       0.210526      0.052632  0.157895  0.105263  0.210526             0.0   
9       0.000000      0.000000  0.000000  0.000000  0.000000             0.0   

  P_Suicide_Act  P_Goal  P_Medical  P_FPS  
0      0.033898     0.0   0.016949    0.0  
1      0.033898     0.0   0.016949    0.0  
2      0.0

In [72]:
df_depression.shape

(4035, 41)

In [73]:
df_depression_post.to_csv('depression_threads.csv')
df_depression.to_csv('depression_comments.csv')